Nom Etudiant 1: <font color='blue'>NOM Prénom</font>
<br>
Nom Etudiant 2: <font color='blue'>NOM Prénom</font>

# TME 2.2 : Projet Detection de motifs - Median String

Les motifs que nous cherchons dans les séquences d'ADN peuvent présenter certaines variations ou mutations. Nous allons développer différents algorithmes pour la recherche de motifs variables (**Partie A**), puis améliorer cette recherche en intégrant une liste étendue de motifs de référence et en développant des fonctions pour l'algorithme Median String (**Partie B**), avant d'appliquer ces approches aux données de ChIP-Seq de _C. glabrata_ (**Partie C**).

## Partie A : Recheche de pattern (motifs) en permettant des variations

1\. Comme dans le TME precedent, nous alons utiliser des données atificielles pour pouvoir tester les algorithmes. Reutilisez les séquences artificielles de taille `t = 41` et le motif de taille `k = 4` sauvegardés le TME précédente.

In [ ]:
seqs_file = 'seqs_artificielles.txt'

# Code
        
for seq in seqs:
    print(seq)

motif_file = "motif.txt"

# Code

print("\nLe motif géneré était :", motif)

2\. Jusqu'à maintenant, nous avons travaillé avec des motifs invariables. Cependant, dans les données biologiques, les motifs peuvent présenter des variations. Créez une fonction pour faire varier un motif selon un certain nombre de positions `v`.

In [ ]:
import random

nuc = ('A', 'C', 'G', 'T')

# Pour obtenir le même résultat que dans le dévelopent de ce TME
#random.seed(seed)

v = 1 # nombre de positions variables dans le motif

def modifierMotif(motif:str, v:int, upper = False):
    """
    Modifie v positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée v: nombre de positions à varier, si 0 le motif sera invariable
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    
    motifM = ""
    
    return motifM

print("Motif original : ", motif)

# Tester la fonction modifierMotif plusieurs fois
for i in range(0, 5):
    print(modifierMotif(motif, 1, upper = False))

3\. Modifiez la fonction `implantMotifs()` de la séance précédente afin d’ajouter la possibilité de varier le motif implanté. Cette fois-ci, considérez que toutes les séquences contiendront un motif (`f1 = 1`). En revanche, seule une proportion des séquences (`f2 = 0.4`) inclura un motif modifié. Pour faciliter leur identification, implantez les motifs en minuscules.

**BONUS :** Implantez un motif modifié dans une proportion de séquences légèrement inférieure (`f1 = 0.9`). Assurez-vous de ne pas ajouter un motif modifié si, par hasard, la fonction `modifierMotif()` génère le même motif que l’original.

In [ ]:
f1 = 1
f2 = 0.4

# Pour obtenir le même résultat que dans le dévelopent de ce TME
#random.seed(seed)

def implantMotifsVar(motifs:str, sequences:list, v:int, f1 = 1, f2 = 0.4):
    """
    Insére un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée séquences : liste de séquences
    entrée v: nombre de positions à varier, si 0 le motif sera invariable
    entrée f : fréquence d'implantation, si 1 toutes les séquences contiendront un motif
    sortie modified_sequences: liste de séquences ayant le motif implanté
    """
    
    modified_sequences = []
            
    return modified_sequences

seqs_implanted = implantMotifsVar(motif, seqs, v, f1, f2)

seqs_implanted

4\. Nous pouvons visualiser les motifs à l'aide des outils de WebLogo : https://weblogo.berkeley.edu/logo.cgi.
Extrayez les motifs implantés dans vos séquences artificielles et visualisez-les à l'aide de WebLogo.
Affichez le logo en utilisant **Markdown**.

In [ ]:
motifs_extraits # Code

print("Motif original : ", motif)

for motif_extrait in motifs_extraits:
        print(motif_extrait)

![Logo](Logo_GCGT.png)

5\. Insérez le motif reverse complémentaire dans les séquences contenant déjà un motif implanté. Si vous avez réussi le **BONUS** de la question 3, vous pouvez insérer ce motif reverse complémentaire avec une fréquence légèrement plus faible, afin de simuler une présence inégale du motif varié reverse complémentaire.

In [ ]:
# Importez vos fonctions

seqs_implanted_rev # Code

seqs_implanted_rev

6\. Sauvegardez vos séquences artificielles implantées avec les motifs variables, ainsi que les motifs variés générés, afin de les réutiliser lors des séances suivantes.

In [ ]:
seqs_impl_file = "seqs_artificielles_impl_var.txt"

motif_file = "motif_var.txt"
        
# Code

## Partie B : Amellioration de la recherche de motifs et développement de l'algorithme Median String 

1\. Précédemment, nous avons recherché des motifs déjà présents dans nos données. Cependant, une meilleur option pourrait être de créer une liste de tous les motifs possibles d'une certaine taille, même si nous ne connaissons pas leur fréquence. Suivant cette logique, combien de motifs (k-mers) de taille `k = 4` pouvons-nous obtenir avec toutes les combinaisons possibles de nucléotides ? Effectuez le calcul avec les fonctions de **Python**. **BONUS :** vous pouvez utiliser **Markdown** en combinaison avec [LaTeX](https://fr.wikipedia.org/wiki/LaTeX), un système de composition de documents qui permet de représenter des expressions mathématiques (par example, $E = mc^2$) pour exprimer votre résultat.

In [ ]:
# Calculer en Python

**BONUS :** Résultat exprimé avec LaTeX

<font color='blue'>
La quantité de k-mers possibles de taille 4 est donnée par: $4^{4}$ = 256
</font>

2\. En utilisant les fonctions de la bibliothèque **itertools**, créez tous les k-mers possibles de taille `k = 4`.

In [ ]:
k = 4

from itertools import product

# Générer tous les k-mers de taille k ayant de AAA... à TTT...
allkmers_k4 # Code

# Visualiser les prémieres 10 k-mers
print(allkmers_k4[0:10])

3\. Éliminez les motifs peu complexes pour éviter les calculs inutiles (motifs ayant plusieurs fois une base répétée, comme **AAAAAAC** ou deux bases répétées comme **ACACACAC**). Réutilisez les fonctions déjà définies (`removeLowComplexeHomo()` et `removeLowComplexeHetero()`) de la séance précédente. Faites attention au type d'objet d'entrée pour chaque fonction. Réfléchissez aux meilleurs paramètres pour filtrer les k-mers selon leur taille. Combien de k-mers restent après avoir supprimé ceux qui sont peu complexes ?

In [ ]:
# Importez vos fonctions

import re

# Réponse BONUS TME2.1 question 6
def removeLowComplexeHetero(motifs:list, n:int, variation = "yes"):
    """
    Enlève les motifs peu complexe ayant n fois un dinucléotide
    entrée motifs: liste de motifs, clé = motif, valeur = fréquence d'observation
    entrée n: taille de répétition de dinucléotide
    entrée variation : string, si "yes" permettre variation d'un nucléotide
    sortie motifsClean: liste de motifs sans les motifs peu complexe
    >>>removeLowComplexeHetero(['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'AGGGTGG', 'TGGCTTA', 'AGAGTAG', 'GCCCCTC'], 3, variation = "yes")
    ['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'TGGCTTA', 'AGAGTAG']
    """
    
    motifsClean = []
    trouve = []

    # Créer une liste de dinucléotides sans mono dinucléotides
    all_di_nuc = [nuc_1 + nuc_2 for nuc_1 in nuc for nuc_2 in nuc if nuc_1 != nuc_2]

    # Pour chaque motif de la liste
    for motif in motifs:
        if variation == "yes":
            # Permettre une varation
            trouve = [nuc_di for nuc_di in all_di_nuc if len(re.findall(re.compile("(?=("+ nuc_di +"))"), motif)) >= n]

        else:
            # Pas permettre de varations
            trouve = [nuc_di for nuc_di in all_di_nuc if len(re.findall(re.compile("(?=("+ nuc_di +")"+ nuc_di[0] +")"), motif)) >= n]

        # Après avoir evalué tous les dinucléotides, évaluer si on a trouvé de dinucleotides
        if len(trouve) == 0:
            # Ajouter juste les motifs que n'ont pas eu des dinucléotides trouves
            motifsClean.append(motif)
        # Vider la liste
        trouve = []

    return motifsClean

# Liste originale
print(len(allkmers_k4))

# Sans homo
allkmers_k4 # Code

print(len(allkmers_k4))

# Sans hetero
allkmers_k4 # Code

print(len(allkmers_k4))

4\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tout d'abord, créez une fonction pour calculer la distance de Hamming.

In [ ]:
def hamDistance(str1:str, str2:str):
    """
    Calcul la distance de Hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de Hamming
    >>>hamDistance("TTGGTAT", "TTGCTAA")
    2
    """
    
    distance = 0
    
    return distance

hamDistance("TTGGTAT", "TTGCTAA")

5\. Postérieurement, créez une fonction permettant d'obtenir la distance totale minimale entre un motif et un ensemble de séquences.

In [ ]:
# Attention à ne pas comparer des séquences en majuscules avec des minuscules.

def totalDistance(motif:str, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: motif à comparer, chaîne de caractères
    entrée sequences: liste de séquences
    entrée k: taille du motif
    sortie total_distance: somme de distance de hamming minimal
    """
    
    total_distance = 0
    
    return total_distance

totalDistance(motif, seqs_implanted, k)

6\. Finalmente, implémentez l'algorithme complet pour tester un liste de motifs et produire un dictionnaire contenant toutes les distances calculées à partir de tous les motifs testés. Trouvez le motif qui donne la distance minimale avec `getTopMotifs()`. Faites attention à ordonner le dictionnaire  dans le bon sens (croissant ou decroissant).

In [ ]:
def medianStringSearch(sequences, kmersV):
    """
    Implement l'algorithme MedianStringSearch
    entrée séquences : liste de séquences
    entrée kmersV: Liste de Kmers à chercher
    sortie motif_dist_dict: un dictionnaire contenant les motifs et leurs distances
    """
    
    motif_dist_dict = {}
    
    return motif_dist_dict

In [ ]:
# Importez vos fonctions

seqs_implanted_motifs # Code

seqs_implanted_motifs_top # Code

print(seqs_implanted_motifs_top)

Avez-vous trouvé le motif implanté ? Parfois, le motif implanté n'est pas le plus fréquent. Vous pouvez explorer le dictionnaire obtenu avec la fonction  `searchGivenMotif()`.

In [ ]:
# Importez vos fonctions
  
# Code 

7\. Rassemblez les résultats de l'algoritme MedianStringSearch sur les séquences du brin complementaire en réutilisant la fonction `gatherRevCompMotifs()`. Combien de motifs restent après avoir fusionné le dictionnaire ? Affichez le nouveau résultat.

In [ ]:
# Importez vos fonctions

print("Avant rassemblage :")

print(len(seqs_implanted_motifs))

seqs_implanted_motifs_gat # Code

print("Après rassemblage :")

print(len(seqs_implanted_motifs_gat))

seqs_implanted_motifs_gat_top # Code

print(seqs_implanted_motifs_gat_top)

## Partie C : Recherche de motifs variable sur vos données

1\. Utilisez le fichier "Sequence_by_Peaks_##.fasta", contenant les régions de peaks identifiées par ChIP-Seq, où se trouve probablement le facteur de transcription recherché. Appliquez l'algorithme Median String pour détecter les motifs Il faut bien evidement enlever les motifs peu complexe. Créez et testez different tailles de motifs. N'oubliez pas de rassembler le dictionnaire obtenu.

In [ ]:
k = 7 # tester d'autres tailles k = 6, k = 8, etc.

m = 5

n = 2

from motifsSearch import readFasta

peaks # Code

allkmers_k7 # Code

# Liste originel
print(len(allkmers_k7))

allkmers_k7 # Code

# Sans homo
print(len(allkmers_k7))

allkmers_k7 # Code

# Sans hetero
print(len(allkmers_k7))

In [ ]:
peaks_motifs # Code

peaks_motifs_top # Code

print("Top motifs :\n", peaks_motifs_top)

2\. Parfois, la TATA box, un type de motif général présent dans les promoteurs de gènes, dépasse le seuil établi pour filtrer les homopolymères et hétéropolymères. Cependant, ces TATA box constituent des motifs généraux plutôt que spécifiques. Une approche "naïve" pour les éliminer consiste à filtrer les motifs contenant une forte proportion de nucléotides T et A. Créez une fonction qui supprime les motifs enrichis en T et A, en veillant à ne les retirer que si les deux nucléotides sont présents.

In [ ]:
p = 0.7

def removeTARich(motifs:list, p:int):
    """
    Enlève les motifs contenant > p de T et A en combination
    entrée motifs: liste de motifs
    entrée p: proportion de nucleótides T et A
    sortie motifsClean: liste de motifs sans les motifs TA rich
    """

    motifsClean = []

    return motifsClean

test_motifs_TA = ['ATTTTCA', 'TAATTTA', 'AAATTAT', 'TCTACGA', 'ACGCATT', 'ACAAGGT', 'GAAATTA', 'TTGTTGT', 'TATAGCA', 'TAGCCTT']

removeTARich(test_motifs_TA, p)

3\. Retestez après avoir enlevé les motifs enrichis en TA.

In [ ]:
allkmers_k7 = removeTARich(allkmers_k7, p)

peaks_motifs # Code

peaks_motifs_top # Code

print("Top motifs :\n", peaks_motifs_top)

**NOTES :** Pensez à developer une fonction pour créer un liste de motifs de manière automatique. Aussi, appliquez les stratégies appris dans ce TME à l'algorithme de la Hash Table :

* Utilisation d'une liste étendue de motifs de référence.
* Suppression des séquences enrichies en T et A.